In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install mahaNLP==0.5 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.2 MB/s eta 0:00:00


In [3]:
!gdown --id 199Eo_tf8_7U-CPjVm74F4ijk-8lahpQh

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=199Eo_tf8_7U-CPjVm74F4ijk-8lahpQh
To: /content/goemotions_subset_train_new_part1.csv
100% 3.44M/3.44M [00:00<00:00, 223MB/s]


In [4]:
!pip install gdown -q

In [5]:
import mahaNLP
import pandas as pd
import torch
import random
import re
import gdown

In [6]:
# data preprocessing:
# removing non marathi text 
import re
def remove_non_marathi_goemotions(text):
    i= re.sub(r"http\S+", "", text)
    i= re.sub(r"pic\S+", "", i)
    i= re.sub(r"#", "", i)
    i= re.sub(r"lokmat\S+", "", i)
    i= re.sub(r"@\S+", "", i)
    i= re.sub(r"fb\S+", "", i)
    i= re.sub(r"twitter\S+", "", i)
    i= re.sub(r"bit\S+", "", i)    
    # i= re.sub(r"['\"!-./,;\[\]?]", "", i)  # during augmenting, keep the necessary punctuations [.?!'"] in the sentence.  
    i= re.sub(r"[\[\]]", "", i)    
    i= re.sub(r'[a-zA-Z0-9]', "", i)    
    i=re.split(r'[\s\n]+', i.strip())
    return(' '.join(i))

In [7]:
df = pd.read_csv('/content/goemotions_subset_train_new_part1.csv')
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,text,three_point,marathi_text
0,0,0,13178,"This is so hard to sit through, I can't believe the swearing from this dood, it makes it all the more unbearable",-1,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर विश्वास बसत नाही, हे सर्व अधिक असह्य करते"
1,1,1,39432,Because idiots in this sub care too much about what other people have on their screen? It only tells me that too many people on here are narcissists.,-1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर काय आहे याची खूप काळजी घेतात? हे फक्त मला सांगते की येथे बरेच लोक नार्सिसिस्ट आहेत.
2,3,3,30127,Water is not a human rights. That's absurd.,-1,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.
3,4,4,10283,The thin side and endlines are really throwing me off.,-1,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.
4,5,5,2930,If there is no goal on this powerplay it will be difficult not to be hopeless,-1,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न होणे कठीण होईल
...,...,...,...,...,...,...
12496,24994,24994,48054,Women should have all the same rights as men. That does not include ending another human’s life.,1,स्त्रियांना पुरुषांसारखेच सर्व अधिकार मिळाले पाहिजेत. त्यात दुसऱ्या माणसाचे जीवन संपवणे समाविष्ट नाही.
12497,24996,24996,32527,This guy has some massive self control after the fact. I felt like getting back and punching that Croc.,1,या व्यक्तीकडे वस्तुस्थितीनंतर काही मोठ्या प्रमाणात आत्मनियंत्रण आहे. मला परत आल्यासारखं वाटलं आणि त्या क्रोकला ठोसा मारावा.
12498,24998,24998,55767,As funny as a sinking school bus.,1,बुडणाऱ्या स्कूल बससारखे मजेदार.
12499,24999,24999,39972,the best places are immigrant restaurants. cheap and delicious food. the better ones are outside of downtown.,1,सर्वोत्तम ठिकाणे स्थलांतरित रेस्टॉरंट्स आहेत. स्वस्त आणि स्वादिष्ट अन्न. जे चांगले आहेत ते डाउनटाउनच्या बाहेर आहेत.


In [8]:
df.drop(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'text'], axis=1, inplace=True)
dict1 = {'marathi_text': 'tweet', 'three_point': 'label'}
df.rename(columns=dict1, inplace=True)
df = df.iloc[:, [1, 0]]

In [9]:
for i in range(len(df)):
  df['tweet'][i] = remove_non_marathi_goemotions(df['tweet'][i])
df = df[df['tweet']!='']

<ipython-input-9-8d6e57f33d8a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet'][i] = remove_non_marathi_goemotions(df['tweet'][i])


In [10]:
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
df = df.reset_index(drop=True)

<ipython-input-10-ca293342285f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)
<ipython-input-10-ca293342285f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [11]:
df.head()

,tweet,label
0,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर विश्वास बसत नाही, हे सर्व अधिक असह्य करते",-1
1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर काय आहे याची खूप काळजी घेतात? हे फक्त मला सांगते की येथे बरेच लोक नार्सिसिस्ट आहेत.,-1
2,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.,-1
3,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.,-1
4,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न होणे कठीण होईल,-1


In [12]:
from mahaNLP.tokenizer import Tokenize
tokenizer = Tokenize()
from mahaNLP.tagger import EntityRecognizer
ner_tagger = EntityRecognizer()
from mahaNLP.maskFill import MaskPredictor
mask_predictor = MaskPredictor()

In [13]:
list_augmented_sentences = []

def mask_NER_entities_in_text_parallel(text):
    # getting the NER tokens from the sentence
    df_ner = ner_tagger.getTokenLabels(text)

    # converting the NER tokens into a list, excluding those from the 'Other' category
    ner_tokens = list(df_ner.loc[df_ner['entity_group'] != 'Other', 'word'])
    # print(ner_tokens)

    

    for token in ner_tokens:
      text = text.replace(token, token+" ", 1)
    # print(text)

    for token in ner_tokens:
      if token not in text:
        ner_tokens.remove(token)

    # for token in ner_tokens:
    #   print(mask_predictor.predictMask(text.replace(token, '[MASK] ', 1))['token_str'][0])

    # replace each NER token with [MASK], get the predicted word, and store in a list
    masked_prediction_list = [
        mask_predictor.predictMask(text.replace(token, '[MASK] ', 1))['token_str'][0]
        if mask_predictor.predictMask(text.replace(token, '[MASK] ', 1))['token_str'][0] != token
        else mask_predictor.predictMask(text.replace(token, '[MASK] ', 1))['token_str'][1]
        for token in ner_tokens
    ]

    # split the sentence into words and replace each NER token with its corresponding predicted word
    words = text.split()
    for i, token in enumerate(words):
        if token in ner_tokens:
            words[i] = masked_prediction_list[ner_tokens.index(token)]

    # join the words back into a sentence and return
    return " ".join(words)

In [14]:
df['tweet'][800]

'मी ड्रायव्हिंग करत आहे आणि मी ओरडणे थांबवू शकत नाही मला कोणाशी तरी सॉरी बोलणे आवश्यक आहे'

In [15]:
mask_NER_entities_in_text_parallel('सर्वोच्च न्यायालयाचे निर्देश डावलून पुणे पोलिसांनी प्रा आनंद तेलतुंबडेंना अटक केली मात्र न्यायालयाने त्यांची अटक बेकायदेशीर ठरवून सरकारला जोरदार चपराक लगावली आहे कायदा हातात घेणाया पोलीस अधिकायांवर तात्काळ कारवाई केली पाहिजे ')

'उच्च न्यायालयाच्या निर्देश डावलून मुंबई पोलिसांनी आज . यांना अटक केली मात्र न्यायालयाने त्यांची अटक बेकायदेशीर ठरवून सरकारला जोरदार चपराक लगावली आहे कायदा हातात घेणाया पोलीस अधिकायांवर तात्काळ कारवाई केली पाहिजे'

In [16]:
df_augmented = pd.DataFrame(columns=df.columns)

In [17]:
df_augmented

,tweet,label


In [18]:
from tqdm import tqdm
for i in tqdm(range(len(df))):
    try:
      row = df.iloc[i]
      j = re.sub(r'[\u200d]+', '', row['tweet'])
      tweet_augmented = mask_NER_entities_in_text_parallel(j)
      row_augmented = row.copy()
      row_augmented['tweet_augmented'] = tweet_augmented
      df_augmented = df_augmented.append(row_augmented, ignore_index=True)
    except Exception:
      continue

Streaming output truncated to the last 5000 lines.
 80%|███████▉  | 9992/12491 [51:34<12:41,  3.28it/s]<ipython-input-18-edee8c5dc2ea>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_augmented = df_augmented.append(row_augmented, ignore_index=True)
 80%|████████  | 9993/12491 [51:34<11:12,  3.72it/s]<ipython-input-18-edee8c5dc2ea>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_augmented = df_augmented.append(row_augmented, ignore_index=True)
 80%|████████  | 9994/12491 [51:34<09:52,  4.21it/s]<ipython-input-18-edee8c5dc2ea>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_augmented = df_augmented.append(row_augmented, ignore_index=True)
 80%|████████  | 9995/12491 [51:34<09:00,  4.62it/s]<ipython-input-18-edee8c

In [19]:
df_augmented.drop(columns=['tweet'], axis=1, inplace=True)

In [20]:
len(df_augmented)

12491

In [21]:
df_augmented = df_augmented.rename(columns={'tweet_augmented': 'tweet'})
# df = pd.concat([df, df_augmented])
df = df.append(df_augmented, ignore_index = True)
df = df.drop_duplicates()
df.dropna(inplace=True)
df = df.reset_index(drop=True)
df.to_csv('/content/gdrive/MyDrive/goemotions_parallel_masking_ner_augmented.csv', index=False)

<ipython-input-21-839e14077264>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_augmented, ignore_index = True)


In [22]:
len(df)

14521